# Data Cleaning

This dataset is quite uncleand and unusable in its current state.

This notebook will be used to clean the data and to annotate my progress (for future reference), then to do the EDA and a recommendation system within a new notebook.

### ** REMEMBER TO EXTRACT CODE INTO AN IDE TO GET A .py SCRIPT **
#### Ask Shanghua what's the best way to do this


In [340]:
# Data manipulation & cleaning
## Since we're just cleaning, we won't require many libraries
import pandas as pd
import numpy as np

In [348]:
# Import data
df = pd.read_csv(r"C:\Users\ssc44611\Documents\L4 Projects\4. Stean Store (Personal Proj) - EDA & ML\steam_app_data.csv")
df_copy1 = df.copy()
df_copy2 = df.copy()

In [317]:
# Saving a smaller, snipped dataset - to visualise the unclean data on github (Original dataset is too large)

#df_copy_newsave = df.copy()
#df_copy_newsave = df_copy_newsave.head(50)
#df_copy_newsave.to_csv(r"C:\Users\ssc44611\Documents\L4 Projects\2. Steam (Personal Proj) - EDA & ML\steam_app_data_PREVIEW")


## Original = 286,616 KB / 286.616 MB ✗
## Preview (New) = 404 KB / 0.404 MB ✓
### GitHub Minimum = 25 MB 

In [318]:
df.head(2)

,type,name,steam_appid,required_age,is_free,controller_support,dlc,detailed_description,about_the_game,short_description,...,categories,genres,screenshots,movies,recommendations,achievements,release_date,support_info,background,content_descriptors
0,game,Counter-Strike,10,0.0,False,NaN,NaN,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,...,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 65735},{'total': 0},"{'coming_soon': False, 'date': '1 Nov, 2000'}","{'url': 'http://steamcommunity.com/app/10', 'e...",https://steamcdn-a.akamaihd.net/steam/apps/10/...,"{'ids': [2, 5], 'notes': 'Includes intense vio..."
1,game,Team Fortress Classic,20,0.0,False,NaN,NaN,One of the most popular online action games of...,One of the most popular online action games of...,One of the most popular online action games of...,...,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 2802},{'total': 0},"{'coming_soon': False, 'date': '1 Apr, 1999'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/20/...,"{'ids': [2, 5], 'notes': 'Includes intense vio..."


In [319]:
# Printing count of rows & columns
print(f"There are {df.shape[0]} rows")
print(f"There are {df.shape[1]} columns")

There are 29235 rows
There are 39 columns


## Data Cleaning - Initial Processing

This section of the cleaning process will focus on the overall dataset - iterating through the columns. 

As we progress into the data cleansing, we will look at the dataset in a more microscopic view.

In [320]:
# Printing null values for each column
na_count = df.isnull().sum()
na_count

type                         149
name                           1
steam_appid                    0
required_age                 149
is_free                      149
controller_support         23237
dlc                        24260
detailed_description         175
about_the_game               175
short_description            175
fullgame                   29235
supported_languages          163
header_image                 149
website                     9983
pc_requirements              149
mac_requirements             149
linux_requirements           149
legal_notice               19168
drm_notice                 29077
ext_user_account_notice    28723
developers                   264
publishers                   149
demos                      27096
price_overview              3712
packages                    3370
package_groups               149
platforms                    149
metacritic                 26254
reviews                    23330
categories                   714
genres    

In [321]:
df_nona = df_copy1.dropna()
df_nona

,type,name,steam_appid,required_age,is_free,controller_support,dlc,detailed_description,about_the_game,short_description,...,categories,genres,screenshots,movies,recommendations,achievements,release_date,support_info,background,content_descriptors


Dropping the null rows without actually looking a little deeper, will mean we just of the entire dataset. 

Since there's a significant amount of null values - essentially a null value for every row - we will have to create a specific criteria of the data with wish to remove

We will look to drop any columns that contain 50% or more null values, relative to the total row count. 

### 1. Columns with >= 50% Null Rows

In [322]:
# Finding columns with >= 50% null values
value_threshold = df.shape[0] // 2 # 50% of total row count, if a columns has greater null values than 50%, we will drop it

print(f"We will need to drop columns with (>=) {value_threshold} null values") # Printing our column drop criteria

We will need to drop columns with (>=) 14617 null values


In [323]:
# Looking at what columns we need to drop 
drop_cols = df.columns[na_count >= value_threshold] # Get the columns that have (null value count >= 50% of total row count)

print(f"We will need to drop: {list(drop_cols)} - (10 columns).") # Printing what columns we need to mention when dropping.


We will need to drop: ['controller_support', 'dlc', 'fullgame', 'legal_notice', 'drm_notice', 'ext_user_account_notice', 'demos', 'metacritic', 'reviews', 'recommendations'] - (10 columns).


In [349]:
# Dropping columns with an abundance of null values
df = df.drop(drop_cols, axis=1)

print('We now have:')
print(f'- {df.shape[0]} rows.')
print(f'- {df.shape[1]} columns.')

We now have:
- 29235 rows.
- 29 columns.


### 2. 'Type' Of Content

Here we will look through the 'Type' column, for any missing data.

In [325]:
# Let's look at the "Type" columns, for any media without a "Type" - we can drop.

print(df['type'].value_counts(dropna=False)) # Without (dropna=False), the NaN rows won't print

print('---------------------------')
print('Amount of rows to drop:', df[df['type'].isnull()].shape[0]) # Printing the amount of NaN rows to drop
print('---------------------------')

df[df['type'].isnull()].head(2) # Looking a preview of the rows with missing data

game    29086
NaN       149
Name: type, dtype: int64
---------------------------
Amount of rows to drop: 149
---------------------------


,type,name,steam_appid,required_age,is_free,detailed_description,about_the_game,short_description,supported_languages,header_image,...,platforms,categories,genres,screenshots,movies,achievements,release_date,support_info,background,content_descriptors
26,NaN,Half-Life: Opposing Force,852,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
147,NaN,Half-Life: Opposing Force,4330,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


It is important to note that these rows have no other values. Other than that, the column provides no other value - every other entity is a game, so we can drop it.

In [350]:
# Dropping 'Type' columns
df = df.drop('type', axis=1)

print('We now have:')
print(f'- {df.shape[0]} rows')
print(f'- {df.shape[1]} columns')

We now have:
- 29235 rows
- 28 columns


### 'Name' Of Content

Here we will look through the 'Name' column - looking for any rows with missing names.

| - Allows for us to call a boolean method, looking for null values, aswell as any rows with 'none' name. 

In [327]:
no_name = df[(df['name'].isnull()) | (df['name'] == 'none')]
no_name

,name,steam_appid,required_age,is_free,detailed_description,about_the_game,short_description,supported_languages,header_image,website,...,platforms,categories,genres,screenshots,movies,achievements,release_date,support_info,background,content_descriptors
4918,none,339860,0.0,False,NaN,NaN,NaN,English<strong>*</strong><br><strong>*</strong...,https://steamcdn-a.akamaihd.net/steam/apps/339...,NaN,...,"{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '25', 'description': 'Adventure'}, {'i...",NaN,NaN,"{'total': 3, 'highlighted': [{'name': 'Master ...","{'coming_soon': False, 'date': '27 Feb, 2015'}","{'url': '', 'email': ''}",NaN,"{'ids': [], 'notes': None}"
6779,none,385020,0.0,False,- discontinued - (please remove),- discontinued - (please remove),- discontinued - (please remove),"English, French, Italian, German, Spanish - Sp...",https://steamcdn-a.akamaihd.net/steam/apps/385...,NaN,...,"{'windows': True, 'mac': True, 'linux': True}","[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '4', 'description': 'Casual'}, {'id': ...",NaN,NaN,{'total': 0},"{'coming_soon': False, 'date': '4 Nov, 2015'}","{'url': '', 'email': ''}",NaN,"{'ids': [], 'notes': None}"
7235,NaN,396420,0.0,True,Spookeningは3Dの恐怖ゲームで、あなたは毎夜に死んでゴーストとして復活します。<b...,Spookeningは3Dの恐怖ゲームで、あなたは毎夜に死んでゴーストとして復活します。<b...,Spookeningは3Dの恐怖ゲームで、あなたは毎夜に死んでゴーストとして復活します。 村...,NaN,https://steamcdn-a.akamaihd.net/steam/apps/396...,NaN,...,"{'windows': True, 'mac': False, 'linux': False}",NaN,NaN,NaN,NaN,{'total': 0},"{'coming_soon': False, 'date': '1 Nov, 2016'}","{'url': '', 'email': ''}",NaN,"{'ids': [], 'notes': None}"
7350,none,398970,0.0,False,NaN,NaN,NaN,English<strong>*</strong><br><strong>*</strong...,https://steamcdn-a.akamaihd.net/steam/apps/398...,NaN,...,"{'windows': True, 'mac': True, 'linux': True}","[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '25', 'description': 'Adventure'}, {'i...",NaN,NaN,"{'total': 35, 'highlighted': [{'name': ""They'v...","{'coming_soon': False, 'date': '5 Nov, 2015'}","{'url': '', 'email': ''}",NaN,"{'ids': [], 'notes': None}"


In [328]:
print(f'We will need to drop: {no_name.shape[0]} rows')

We will need to drop: 4 rows


In [351]:
# Dropping excess rows
df = df.drop(index = [4918, 6779, 7235, 7350], axis=0) # Dropping the specific rows via each row's index

print(f'We now have: {df.shape[0]} rows')

We now have: 29231 rows


### Duplicated Rows

Here we will look at any rows that are duplicates. Since they don't add any further value, it is safe to drop them.

In [345]:
# Looking for any duplicates
dupes = df[df.duplicated()] # adding df[...] allows us to return the values in a table format
dupes

,name,steam_appid,required_age,is_free,detailed_description,about_the_game,short_description,supported_languages,header_image,website,...,platforms,categories,genres,screenshots,movies,achievements,release_date,support_info,background,content_descriptors
31,SiN Episodes: Emergence,1300,0.0,False,"You are John Blade, commander of HardCorps, an...","You are John Blade, commander of HardCorps, an...","You are John Blade, commander of HardCorps, an...","English, Russian, French",https://steamcdn-a.akamaihd.net/steam/apps/130...,NaN,...,"{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 0},"{'coming_soon': False, 'date': '10 May, 2006'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/130...,"{'ids': [], 'notes': None}"
32,SiN Episodes: Emergence,1300,0.0,False,"You are John Blade, commander of HardCorps, an...","You are John Blade, commander of HardCorps, an...","You are John Blade, commander of HardCorps, an...","English, Russian, French",https://steamcdn-a.akamaihd.net/steam/apps/130...,NaN,...,"{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 0},"{'coming_soon': False, 'date': '10 May, 2006'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/130...,"{'ids': [], 'notes': None}"
356,Jagged Alliance 2 Gold,1620,0.0,False,<p>The small country of Arulco has been taken ...,<p>The small country of Arulco has been taken ...,The small country of Arulco has been taken ove...,English,https://steamcdn-a.akamaihd.net/steam/apps/162...,http://www.jaggedalliance2.com/,...,"{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}]","[{'id': '2', 'description': 'Strategy'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 0},"{'coming_soon': False, 'date': '6 Jul, 2006'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/162...,"{'ids': [], 'notes': None}"
493,Crazy Machines 1.5,18430,0.0,False,Create Your Own Zany &quot;Rube Goldberg&quot;...,Create Your Own Zany &quot;Rube Goldberg&quot;...,Create Your Own Zany &quot;Rube Goldberg&quot;...,English,https://steamcdn-a.akamaihd.net/steam/apps/184...,NaN,...,"{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}]","[{'id': '4', 'description': 'Casual'}, {'id': ...","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 0},"{'coming_soon': False, 'date': '12 Dec, 2008'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/184...,"{'ids': [], 'notes': None}"
494,Crazy Machines 1.5,18430,0.0,False,Create Your Own Zany &quot;Rube Goldberg&quot;...,Create Your Own Zany &quot;Rube Goldberg&quot;...,Create Your Own Zany &quot;Rube Goldberg&quot;...,English,https://steamcdn-a.akamaihd.net/steam/apps/184...,NaN,...,"{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}]","[{'id': '4', 'description': 'Casual'}, {'id': ...","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 0},"{'coming_soon': False, 'date': '12 Dec, 2008'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/184...,"{'ids': [], 'notes': None}"
530,F.E.A.R.,21090,18.0,False,Be the hero in your own cinematic epic of acti...,Be the hero in your own cinematic epic of acti...,Experience the original F.E.A.R. along with F....,English,https://steamcdn-a.akamaihd.net/steam/apps/210...,https://wbgamessupport.wbgames.com/hc/en-us,...,"{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}]","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 0},"{'coming_soon': False, 'date': '21 May, 2010'}",{'

In [335]:
print(f'There are {dupes.shape[0]} duplicated rows, we will need to drop them') # Printing amount of values we need to drop

There are 7 duplicated rows, we will need to drop them


In [352]:
df = df.drop(index = [31, 32, 356, 493, 494, 530, 531], axis=0) # Dropping the specific rows via each row's index

print(f'There are now {df.shape[0]} rows')

There are now 29224 rows


Creating a quick checkpoint of current cleaning progess

In [353]:
checkpoint_1 = df.copy()

In [371]:
print(f'There were: *{df_copy1.shape[0]} rows*, and * {df_copy1.shape[1]} columns*')

print('---------------------------------------------')

print(f'There are now: *{df.shape[0]} rows*, and *{df.shape[1]} columns*')

There were: *29235 rows*, and * 39 columns*
---------------------------------------------
There are now: *29224 rows*, and *28 columns*
